In [25]:
# load artifacts (from disk)
import json, pandas as pd
from chessrl.eval.adapters import PolicyMapAgent, GreedyFromVAgent, MCTSAgent
from chessrl.utils.helpers import make_env_factory
from chessrl.eval.evaluator import evaluate
from chessrl.algorithms.mcts import MCTS

TB_PATH = "../../../syzygy-tables/"

# # 1) If you saved a policy map (fen->uci)
# policy_map = {json.loads(l)["fen"]: json.loads(l)["uci"] for l in open("artifacts/policies/vi_krk_greedy.jsonl")}
# vi_agent = PolicyMapAgent(policy_map)

# # 2) If you saved values (fen->V) and prefer greedy-from-V at test-time
# V = pd.read_parquet("artifacts/values/vi_krk_values.parquet").set_index("fen")["V"].to_dict()
env_factory = make_env_factory(TB_PATH)
# vi_v_agent = GreedyFromVAgent(V, env_factory)

# 3) MCTS (no artifact)
mcts_agent = MCTSAgent(MCTS(iterations=1000, seconds=20.0))


In [20]:

# dtm oracle (e.g., from your CSV where DTM ≈ abs(DTZ) in KRK/KQK/KBBK)
dtz_map = pd.read_csv(TB_PATH + "/krk_dtz.csv").set_index("fen")["dtz"].to_dict()
dtm_oracle = lambda fen: abs(int(dtz_map[fen]))


In [26]:

from chessrl.utils.helpers import make_env_factory, optimal_moves_syzygy, dtm_from_dtz_map

dtm_oracle = dtm_from_dtz_map(dtz_map)

env_factory = make_env_factory(TB_PATH)
opt_moves   = optimal_moves_syzygy(TB_PATH)

# evaluate (same function for all)
test_fens = [ "8/8/8/8/8/8/k7/2K4R w - - 0 1", "8/8/8/8/8/8/k7/2K2R2 w - - 0 1", "8/8/8/8/8/8/k7/2K4R w - - 0 1" ] 
# df_vi     = evaluate(vi_agent.act,   test_fens, env_factory, dtm_oracle)
# df_vi_v   = evaluate(vi_v_agent.act, test_fens, env_factory, dtm_oracle)
df_mcts   = evaluate(mcts_agent.act, test_fens, env_factory, dtm_oracle, optimal_moves=opt_moves, budget=1000)



In [27]:
df_mcts

,fen,plies_policy,dtm_oracle,gap,success,ms_per_move,top1,budget
0,8/8/8/8/8/8/k7/2K4R w - - 0 1,3,3,0,1,6061.486182,1,1000
1,8/8/8/8/8/8/k7/2K2R2 w - - 0 1,3,3,0,1,6108.209548,1,1000
2,8/8/8/8/8/8/k7/2K4R w - - 0 1,3,3,0,1,5588.644659,1,1000
